In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from list_prompt_family import ListPromptFamily
from prompt_registry import PROMPT_REGISTRY, ALL_PROMPTS
from wrap_registry import WRAP_REGISTRY
import numpy as np
import torch
from dotenv import load_dotenv
import os
from transformer_lens import HookedTransformer


/Users/johnwu/.pyenv/versions/3.9.1/lib/python3.9/site-packages/torch/onnx/_internal/_beartype.py:36: UserWarning: unhashable type: 'list'
  warnings.warn(f"{e}")


In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `transformerlens` has been saved to /Users/johnwu/.cache/huggingface/stored_tokens
Your token has been saved to /Users/johnwu/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
MODEL_NAME = "Phi-2"
torch.mps.empty_cache()
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    device="mps",  
)           

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model Phi-2 into HookedTransformer


In [4]:
def gen_list_comprehension_prompt(start: int, end: int, step: int):
    return f'What is the output of [i for i in range({start}, {end}, {step})]. Only output a list, no other information. List:', [f'{i}' for i in range(start, end, step)]

def gen_find_index_prompt(elm, correct):
    return lambda lst: f"What is the output of the following python code {lst}.index({elm})?\nAnswer:", correct

prompts = []

lsts = []
for i in range(5): 
    lst = [1, 1, 1, 1, 1]
    lst[i] = 2
    lsts.append(lst)
    prompt_fn, correct = gen_find_index_prompt(2, i)
    prompt = prompt_fn(lst)
    prompts.append(prompt_fn)
    # answer = model.generate(prompt, temperature=0.0, top_k=0, max_new_tokens=1)
    # print(model.to_str_tokens(answer))
    
pairs = []
answers = []
for i in range(5):
    for j in range(5):
        if i != j:
            pairs.append({'clean': prompts[i](lsts[i]), 'corrupted': prompts[i](lsts[j])})
            answers.append({'clean': model.to_single_token(f"{i}"), 'corrupted': model.to_single_token(f"{j}")})
for pair, answer in zip(pairs, answers): 
    print(pair)
    print(answer)

{'clean': 'What is the output of the following python code [2, 1, 1, 1, 1].index(2)?\nAnswer:', 'corrupted': 'What is the output of the following python code [1, 2, 1, 1, 1].index(2)?\nAnswer:'}
{'clean': 15, 'corrupted': 16}
{'clean': 'What is the output of the following python code [2, 1, 1, 1, 1].index(2)?\nAnswer:', 'corrupted': 'What is the output of the following python code [1, 1, 2, 1, 1].index(2)?\nAnswer:'}
{'clean': 15, 'corrupted': 17}
{'clean': 'What is the output of the following python code [2, 1, 1, 1, 1].index(2)?\nAnswer:', 'corrupted': 'What is the output of the following python code [1, 1, 1, 2, 1].index(2)?\nAnswer:'}
{'clean': 15, 'corrupted': 18}
{'clean': 'What is the output of the following python code [2, 1, 1, 1, 1].index(2)?\nAnswer:', 'corrupted': 'What is the output of the following python code [1, 1, 1, 1, 2].index(2)?\nAnswer:'}
{'clean': 15, 'corrupted': 19}
{'clean': 'What is the output of the following python code [1, 2, 1, 1, 1].index(2)?\nAnswer:', 

In [5]:
clean_tokens = model.to_tokens([pair['clean'] for pair in pairs])
corrupted_tokens = model.to_tokens([pair['corrupted'] for pair in pairs])

answer_token_indices = torch.tensor([[answers[i]['clean'], answers[i]['corrupted']] for i in range(len(answers))])

In [7]:
def get_logit_diff(logits, answer_token_indices=answer_token_indices):
    if len(logits.shape)==3:
        logits = logits[:, -1, :]
    correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    return (correct_logits - incorrect_logits).mean()

clean_logits, clean_cache = model.run_with_cache(clean_tokens)
torch.mps.empty_cache()
print(torch.mps.current_allocated_memory())
# corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)

# clean_logit_diff = get_logit_diff(clean_logits, answer_token_indices).item()
# print(f"Clean logit diff: {clean_logit_diff:.4f}")

# corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_token_indices).item()
# print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")


RuntimeError: MPS backend out of memory (MPS allocated: 27.21 GB, other allocations: 672.00 KB, max allowed: 27.20 GB). Tried to allocate 5.47 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [16]:
print(clean_logits)

print(clean_cache.items())

tensor([[[ 7.9192, 11.4271, 13.7004,  ..., -3.7537, -3.7544, -3.7543],
         [ 6.8309,  4.9825,  1.6050,  ..., -3.3869, -3.3868, -3.3861],
         [ 2.8343,  2.0546, -0.1482,  ..., -3.0613, -3.0606, -3.0603],
         ...,
         [ 6.5264, 12.5410, 14.7584,  ..., -0.6470, -0.6481, -0.6498],
         [ 8.8152,  7.9323,  3.3557,  ..., -1.3492, -1.3488, -1.3499],
         [ 4.6517,  6.4557,  6.7365,  ..., -2.2235, -2.2232, -2.2245]],

        [[ 7.9192, 11.4271, 13.7004,  ..., -3.7537, -3.7544, -3.7543],
         [ 6.8309,  4.9825,  1.6050,  ..., -3.3869, -3.3868, -3.3861],
         [ 2.8343,  2.0546, -0.1482,  ..., -3.0613, -3.0606, -3.0603],
         ...,
         [ 6.5264, 12.5410, 14.7584,  ..., -0.6470, -0.6481, -0.6498],
         [ 8.8152,  7.9323,  3.3557,  ..., -1.3492, -1.3488, -1.3499],
         [ 4.6517,  6.4557,  6.7365,  ..., -2.2235, -2.2232, -2.2245]],

        [[ 7.9192, 11.4271, 13.7004,  ..., -3.7537, -3.7544, -3.7543],
         [ 6.8309,  4.9825,  1.6050,  ..., -3

KeyboardInterrupt: 

In [ ]:
CLEAN_BASELINE = clean_logit_diff
CORRUPTED_BASELINE = corrupted_logit_diff
def ioi_metric(logits, answer_token_indices=answer_token_indices):
    return (get_logit_diff(logits, answer_token_indices) - CORRUPTED_BASELINE) / (CLEAN_BASELINE  - CORRUPTED_BASELINE)

print(f"Clean Baseline is 1: {ioi_metric(clean_logits).item():.4f}")
print(f"Corrupted Baseline is 0: {ioi_metric(corrupted_logits).item():.4f}")

In [ ]:
import transformer_lens.patching as patching
from neel_plotly import line, imshow, scatter

resid_pre_act_patch_results = patching.get_act_patch_resid_pre(model, corrupted_tokens, clean_cache, ioi_metric)
imshow(resid_pre_act_patch_results, 
       yaxis="Layer", 
       xaxis="Position", 
       x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
       title="resid_pre Activation Patching")